# Train models for visual proprioception

Train a regression model for visual proprioception. The input is sensory data (eg. a camera image). This is encoded by a p;predefined sensorprocessing component into a latent representation. What we are training and saving here is a regressor that is mapping the latent representation to the position of the robot (eg. a vector of 6 degrees of freedom).

The specification of this regressor is specified in an experiment of the type "visual_proprioception". Running this notebook will train and save this model.

In [1]:
import sys
sys.path.append("..")
from settings import Config

import pathlib
from pprint import pprint
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
#import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(1)

from visual_proprioception.visproprio_helper import load_demonstrations_as_proprioception_training, get_visual_proprioception_sp
from visual_proprioception.visproprio_models import VisProprio_SimpleMLPRegression

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Loading pointer config file: /home/ssheikholeslami/.config/BerryPicker/mainsettings.yaml
Loading machine-specific config file: /home/ssheikholeslami/SaharaBerryPickerData/settings-sahara.yaml
Using device: cuda


In [2]:
experiment = "visual_proprioception"
# the latent space 128 ones
# run = "vp_aruco_128"
# run = "vp_convvae_128"
# run = "vp_ptun_vgg19_128"
# run = "vp_ptun_resnet50_128"

# the latent space 256 ones
# run = "vp_convvae_256"
run = "vp_ptun_vgg19_256"
# run = "vp_ptun_resnet50_256"

# the VIT
# run = "vit_base"
# run = "vit_huge"
# run = "vit_large"
# run = "vitwb"

exp = Config().get_experiment(experiment, run)
pprint(exp)

sp = get_visual_proprioception_sp(exp, device)


No system dependent experiment file
 /home/ssheikholeslami/SaharaBerryPickerData/experiments-Config/visual_proprioception/vp_convvae_256_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: visual_proprioception/vp_convvae_256 successfully loaded
{'data_dir': PosixPath('/home/ssheikholeslami/SaharaBerryPickerData/experiment_data/visual_proprioception/vp_convvae_256'),
 'encoding_size': 256,
 'epochs': 1000,
 'exp_run_sys_indep_file': PosixPath('/lustre/fs1/home/ssheikholeslami/BerryPicker/src/experiment_configs/visual_proprioception/vp_convvae_256.yaml'),
 'group_name': 'visual_proprioception',
 'loss': 'MSE',
 'name': 'conv-vae-256',
 'output_size': 6,
 'proprioception_input_file': 'train_inputs.pt',
 'proprioception_mlp_model_file': 'proprioception_mlp.pth',
 'proprioception_target_file': 'train_targets.pt',
 'proprioception_test_input_file': 'test_inputs.pt',
 'proprioception_test_target_file': 'test_targets.pt',
 'proprioception_testing_task': 'proprio_regressor_vali

In [3]:
# Create the regression model

model = VisProprio_SimpleMLPRegression(exp)
if exp["loss"] == "MSE":
    criterion = nn.MSELoss()
elif exp["loss"] == "L1":
    criterion = nn.L1Loss()
else:
    raise Exception(f'Unknown loss type {exp["loss"]}')

optimizer = optim.Adam(model.parameters(), lr=0.001)

{'proprioception_mlp_model_file': 'proprioception_mlp.pth', 'proprioception_input_file': 'train_inputs.pt', 'proprioception_target_file': 'train_targets.pt', 'proprioception_test_input_file': 'test_inputs.pt', 'proprioception_test_target_file': 'test_targets.pt', 'encoding_size': 256, 'regressor_hidden_size_1': 64, 'regressor_hidden_size_2': 64, 'output_size': 6, 'epochs': 1000, 'group_name': 'visual_proprioception', 'name': 'conv-vae-256', 'proprioception_training_task': 'proprio_regressor_training', 'proprioception_testing_task': 'proprio_regressor_validation', 'loss': 'MSE', 'sensor_processing': 'ConvVaeSensorProcessing', 'sp_experiment': 'sensorprocessing_conv_vae', 'sp_run': 'proprio_256', 'run_name': 'vp_convvae_256', 'data_dir': PosixPath('/home/ssheikholeslami/SaharaBerryPickerData/experiment_data/visual_proprioception/vp_convvae_256'), 'exp_run_sys_indep_file': PosixPath('/lustre/fs1/home/ssheikholeslami/BerryPicker/src/experiment_configs/visual_proprioception/vp_convvae_256.y

### Load and cache the training data. 
* Iterate through the images and process them into latent encodings. 
* Iterate through the json files describing the robot position
* Save the input and target values into files in the experiment directory. These will act as caches for later runs
* Create the training and validation splits

In [4]:
task = exp["proprioception_training_task"]
proprioception_input_file = pathlib.Path(
    exp["data_dir"], exp["proprioception_input_file"])
proprioception_target_file = pathlib.Path(
    exp["data_dir"], exp["proprioception_target_file"])
tr = load_demonstrations_as_proprioception_training(
    sp, task, proprioception_input_file, proprioception_target_file)
inputs_training = tr["inputs_training"]
targets_training = tr["targets_training"]
inputs_validation = tr["inputs_validation"]
targets_validation = tr["targets_validation"]

# Create DataLoaders for batching
batch_size = 32
train_dataset = TensorDataset(inputs_training, targets_training)
test_dataset = TensorDataset(inputs_validation, targets_validation)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Cameras found: ['dev2']
There are 562 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
this is bcdddddddd
{'actiontype': 'rc-position-target',
 'camera': 'dev2',
 'cameras': ['dev2'],
 'maxsteps': 562,
 'sensorprocessor': <sensorprocessing.sp_conv_vae.ConvVaeSensorProcessing object at 0x7f2e4772b9d0>,
 'source_dir': PosixPath('/home/ssheikholeslami/SaharaBerryPickerData/demonstrations/demos/proprio_regressor_training/2025_03_08__14_38_55'),
 'trim_from': 1,
 'trim_to': 562}
this is z.shape
(561, 256)
this a.shape
(561, 6)
this is z.shape
(561, 256)
this is anorm.shape
(561, 6)
Cameras found: ['dev2']
There are 487 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
this is bcdddddddd
{'actiontype': 'rc-position-target',
 'camera': 'dev2',
 'cameras': ['dev2'],
 'maxsteps': 487,
 'sensorprocessor': <sensorprocessing.sp_conv_vae.ConvVaeSensorProcessing object at 0x7f2e4772b9d0>,
 'source_dir': PosixP

### Perform the training

In [5]:
def train_and_save_proprioception_model(exp):
    """Trains and saves the proprioception model
    """
    modelfile = pathlib.Path(exp["data_dir"],
                         exp["proprioception_mlp_model_file"])
    if modelfile.exists():
        raise Exception(f'Model already trained {modelfile}.')
    num_epochs = exp["epochs"]
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_X, batch_y in train_loader:
            # Forward pass
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}')

    # Evaluate the model
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            test_loss += loss.item()

    test_loss /= len(test_loader)
    print(f'Test Loss: {test_loss:.4f}')
    torch.save(model.state_dict(), modelfile)

In [6]:
# modelfile = pathlib.Path(Config()["explorations"]["proprioception_mlp_model_file"])

#if modelfile.exists():
#    model.load_state_dict(torch.load(modelfile))
#else:
train_and_save_proprioception_model(exp)

Epoch [10/1000], Loss: 0.0226
Epoch [20/1000], Loss: 0.0132
Epoch [30/1000], Loss: 0.0099
Epoch [40/1000], Loss: 0.0084
Epoch [50/1000], Loss: 0.0070
Epoch [60/1000], Loss: 0.0062
Epoch [70/1000], Loss: 0.0053
Epoch [80/1000], Loss: 0.0047
Epoch [90/1000], Loss: 0.0045
Epoch [100/1000], Loss: 0.0037
Epoch [110/1000], Loss: 0.0036
Epoch [120/1000], Loss: 0.0032
Epoch [130/1000], Loss: 0.0034
Epoch [140/1000], Loss: 0.0027
Epoch [150/1000], Loss: 0.0027
Epoch [160/1000], Loss: 0.0025
Epoch [170/1000], Loss: 0.0024
Epoch [180/1000], Loss: 0.0024
Epoch [190/1000], Loss: 0.0021
Epoch [200/1000], Loss: 0.0020
Epoch [210/1000], Loss: 0.0022
Epoch [220/1000], Loss: 0.0019
Epoch [230/1000], Loss: 0.0017
Epoch [240/1000], Loss: 0.0018
Epoch [250/1000], Loss: 0.0017
Epoch [260/1000], Loss: 0.0017
Epoch [270/1000], Loss: 0.0019
Epoch [280/1000], Loss: 0.0016
Epoch [290/1000], Loss: 0.0015
Epoch [300/1000], Loss: 0.0018
Epoch [310/1000], Loss: 0.0016
Epoch [320/1000], Loss: 0.0018
Epoch [330/1000],